In [5]:
!pip install albumentations tensorflow-addons scipy==1.4.1

In [1]:
import datetime
import os
from collections import defaultdict
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from typing import List, Tuple

from albumentations import (
    Compose, Flip, Rotate
)

from data_science.augmented_image_sequence_from_npy import AugmentedImageSequenceFromNpy

import gc

import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.metrics import Accuracy, Precision, Recall

from tensorflow_addons.metrics import FBetaScore, F1Score

import random

pal = sns.color_palette()

import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

root = os.environ.get("ROOT", '/home/jovyan/work')
experiment_name = "basic_cnn_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

big_earth = os.path.join(root, 'data', 'big_earth')
metadata_path = os.path.join(big_earth, 'metadata', 'metadata.csv')
train_path = os.path.join(big_earth, 'npy_image_files')
test_path = train_path

weight_dir = os.path.join(big_earth, 'model', 'model_weights')
log_dir = os.path.join(big_earth, 'model', 'logs', experiment_name)

for dir_to_create in [weight_dir, log_dir]:
    if not os.path.exists(dir_to_create):
        os.makedirs(dir_to_create)

# ModelCheckpoint expects a path
weight_path = os.environ.get("WEIGHT_PATH",
    os.path.join(weight_dir, 'model_' + experiment_name + '_{epoch:02d}-{val_loss:.2f}.hdf5'))
print('weight_path', weight_path)

files = pd.Index(os.listdir(train_path)).str.replace(".npy", "")
df = pd.read_csv(metadata_path)
df = df.set_index('image_prefix', drop=False)
df = df.loc[df.index.intersection(files)]
df['image_prefix'] = (train_path + "/" + df['image_prefix'] + ".npy")

# sanity check paths
print(df.head(5)['image_prefix'].values)

df_cloud_and_shadow = df[(df['has_cloud_and_shadow'] == 1)]
df_no_cloud_or_snow = df[(df['has_cloud_and_shadow'] == 0) & (df['has_snow'] == 0)]
sample_no_cloud_or_snow = df_no_cloud_or_snow.sample(n=len(df_cloud_and_shadow))
print(
    'len(df)', len(df), 
    'len(df_cloud_and_shadow)', len(df_cloud_and_shadow), 
    "len(df_no_cloud_or_snow)", len(df_no_cloud_or_snow),
    'len(sample_no_cloud_or_snow)', len(sample_no_cloud_or_snow), 
     )

# 44 level 3 classes:
# Currently using:
# https://land.copernicus.eu/user-corner/technical-library/corine-land-cover-nomenclature-guidelines/html/
classes = ["Continuous urban fabric", "Discontinuous urban fabric", "Industrial or commercial units",
       "Road and rail networks and associated land", "Port areas", "Airports", "Mineral extraction sites",
       "Dump sites",
       "Construction sites", "Green urban areas", "Sport and leisure facilities", "Non-irrigated arable land",
       "Permanently irrigated land", "Rice fields", "Vineyards", "Fruit trees and berry plantations",
       "Olive groves",
       "Pastures", "Annual crops associated with permanent crops", "Complex cultivation patterns",
       "Land principally occupied by agriculture, with significant areas of natural vegetation",
       "Agro-forestry areas",
       "Broad-leaved forest", "Coniferous forest", "Mixed forest", "Natural grassland", "Moors and heathland",
       "Sclerophyllous vegetation", "Transitional woodland/shrub", "Beaches, dunes, sands", "Bare rock",
       "Sparsely vegetated areas", "Burnt areas", "Glaciers and perpetual snow", "Inland marshes", "Peatbogs",
       "Salt marshes", "Salines", "Intertidal flats", "Water courses", "Water bodies", "Coastal lagoons",
       "Estuaries",
       "Sea and ocean"]

# n_classes = len(classes)

weight_path /home/jovyan/work/data/big_earth/model/model_weights/model_basic_cnn_20200121-203404_{epoch:02d}-{val_loss:.2f}.hdf5
['/home/jovyan/work/data/big_earth/npy_image_files/S2B_MSIL2A_20170801T095029_6_66.npy'
 '/home/jovyan/work/data/big_earth/npy_image_files/S2B_MSIL2A_20180515T094029_40_73.npy'
 '/home/jovyan/work/data/big_earth/npy_image_files/S2B_MSIL2A_20171112T114339_43_30.npy'
 '/home/jovyan/work/data/big_earth/npy_image_files/S2A_MSIL2A_20180527T093041_65_35.npy'
 '/home/jovyan/work/data/big_earth/npy_image_files/S2B_MSIL2A_20180224T112109_16_25.npy']
len(df) 490 len(df_cloud_and_shadow) 6 len(df_no_cloud_or_snow) 437 len(sample_no_cloud_or_snow) 6


In [3]:
def balanced_class_train_test_splits(*dataframes: List[pd.DataFrame]):
    train = []
    valid = []
    test = []

    for dataframe in dataframes:
        random_mask = np.random.rand(len(dataframe))
        train_mask = random_mask < 0.6
        valid_mask = (0.6 <= random_mask) & (random_mask < 0.8)
        test_mask = random_mask >= 0.8

        train.append(dataframe[train_mask])
        valid.append(dataframe[valid_mask])
        test.append(dataframe[test_mask])

    train = pd.concat(train)
    valid = pd.concat(valid)
    test = pd.concat(test)
    
    print("len(train)", len(train), "len(valid)", len(valid), "len(test)", len(test))
    return train, valid, test

def basic_cnn_model(img_shape, n_classes):
    """
    From https://arxiv.org/pdf/1902.06148.pdf

    To this end, we selected a shallow CNN architecture, which consists of three convolutional layers with 32, 32 and
    64 filters having 5 × 5, 5 × 5 and 3 × 3 filter sizes, respectively. We
    added one fully connected (FC) layer and one classification
    layer to the output of last convolutional layer. In all convolution operations, zero padding was used. We also applied
    max-pooling between layers.
    """
    img_inputs = Input(shape=img_shape)
    conv_1 = Conv2D(32, (5, 5), activation='relu')(img_inputs)
    maxpool_1 = MaxPooling2D((2, 2))(conv_1)
    conv_2 = Conv2D(32, (5, 5), activation='relu')(maxpool_1)
    maxpool_2 = MaxPooling2D((2, 2))(conv_2)
    conv_3 = Conv2D(64, (3, 3), activation='relu')(maxpool_2)
    flatten = Flatten()(conv_3)
    dense_1 = Dense(64, activation='relu')(flatten)
    output = Dense(n_classes, activation='sigmoid')(dense_1)

    return Model(inputs=img_inputs, outputs=output)


def pretrained_model(base_model_class, input_shape, output_shape):
    """
    All of the top performers use transfer learning and image augmentation: https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/33559

    Another useful discussion on both topics: https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/36091#202629
    """
    # from https://www.kaggle.com/sashakorekov/end-to-end-resnet50-with-tta-lb-0-93#L321
    base_model = base_model_class(include_top=False, input_shape=input_shape, pooling='avg', weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.25)(x)
    output = Dense(output_shape, activation='sigmoid')(x)
    model = Model(inputs=base_model.inputs, outputs=output)
    model.name = base_model.name

    return model


def train(model, x_train: np.array, y_train: np.array, x_valid: np.array,
          y_valid: np.array, n_epochs, n_classes, batch_size, log_dir, weight_path):
    """
    Based on from https://www.kaggle.com/infinitewing/keras-solution-and-my-experience-0-92664
    """
    print(f'Split train: {len(x_train)}')
    print(f'Split valid: {len(x_valid)}')

    histories = []
    learn_rates = [0.001, 0.0001, 0.00001]
    metrics = [Accuracy(), Precision(), Recall(), F1Score(num_classes=n_classes, average='micro'),
               FBetaScore(num_classes=n_classes, beta=2.0, average='micro')]
    loss = 'binary_crossentropy'
    metric_to_monitor = 'val_loss'

    for learn_rate_num, learn_rate in enumerate(learn_rates):
        print(f'Training model on fold with learn_rate {learn_rate}')
        optimizer = Adam(lr=learn_rate, momentum=0.9)
        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

        verbosity = 0
        callbacks = [
            EarlyStopping(monitor=metric_to_monitor, patience=2, verbose=verbosity),
            ReduceLROnPlateau(monitor=metric_to_monitor, factor=0.5, patience=2, min_lr=0.000001),
            TensorBoard(log_dir, histogram_freq=1),
            ModelCheckpoint(weight_path, monitor=metric_to_monitor, save_weights_only=False, save_best_only=True,
                            verbose=verbosity)
        ]

        # Generators
        train_generator = AugmentedImageSequenceFromNpy(x=x_train, y=y_train, batch_size=batch_size,
                                                        augmentations=AUGMENTATIONS_TRAIN)

        valid_generator = AugmentedImageSequenceFromNpy(x=x_valid, y=y_valid, batch_size=batch_size,
                                                        augmentations=AUGMENTATIONS_TEST)

        history = model.fit_generator(generator=train_generator,
                                      epochs=n_epochs,
                                      steps_per_epoch=len(train_generator),
                                      callbacks=callbacks,
                                      validation_data=valid_generator, validation_steps=len(valid_generator),
                                      shuffle=True, verbose=1)
        histories.append(history)

    # Attempt to avoid memory leaks
    del train_generator
    del valid_generator
    gc.collect()

    return histories


def join_histories(histories):
    full_history = defaultdict(list)

    for history in histories:
        for key, value in history.history.items():
            full_history[key].extend(value)
    return full_history


def graph_model_history(history):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    t = f.suptitle('Basic CNN Performance', fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    max_epoch = len(history['val_loss']) + 1
    epoch_list = list(range(1, max_epoch))
    ax1.plot(epoch_list, history['accuracy'], label='Train Accuracy')
    ax1.plot(epoch_list, history['val_accuracy'], label='Validation Accuracy')
    ax1.set_xticks(np.arange(1, max_epoch, 5))
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    l1 = ax1.legend(loc="best")

    ax2.plot(epoch_list, history['loss'], label='Train Loss')
    ax2.plot(epoch_list, history['val_loss'], label='Validation Loss')
    ax2.set_xticks(np.arange(1, max_epoch, 5))
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    l2 = ax2.legend(loc="best")


def predict(model, weight_dir, x, batch_size, n_classes):
    thresholds = np.array([0.5 for _ in range(n_classes)])

    weight_path = os.path.join(weight_dir, 'weights_kfold_' + str(num_fold) + '.h5')
    model.load_weights(weight_path)

    predict_generator = AugmentedImageSequenceFromNpy(x=x, y=None, batch_size=batch_size,
                                                        augmentations=AUGMENTATIONS_TEST)
    # Generators
    pred_test_probs = model.predict_generator(predict_generator)
    pred_test_labels = pd.DataFrame(pred_test_probs, columns=classes)
    pred_test_labels = pred_test_labels.apply(lambda x: x > thresholds, axis=1)
    # Convert boolean predictions to labels
    pred_test_lables = pred_test_labels.apply(lambda row: ' '.join(row[row].index), axis=1)

    del predict_generator
    gc.collect()

    return pred_test_labels

train, valid, test = balanced_class_train_test_splits(
    *[sample_no_cloud_or_snow, df_cloud_and_shadow])

AUGMENTATIONS_TRAIN = Compose([
    Flip(p=0.5),
    Rotate(limit=(0, 360), p=0.5)
])

AUGMENTATIONS_TEST = Compose([])

len(train) 8 len(valid) 3 len(test) 1


In [ ]:
n_classes = 1

n_epochs = 100
model = basic_cnn_model((120, 120, 3), n_classes=n_classes)

# Test the correctness and speed of loading one batch
batch_size = 128
test_data_len = batch_size * 2

x_train = train['image_prefix'].values
x_valid = valid['image_prefix'].values
x_test = test['image_prefix'].values

y_train = np.random.randint(0, 2, (len(train), n_classes))
y_valid = np.random.randint(0, 2, (len(valid), n_classes))
y_test = np.random.randint(0, 2, (len(test), n_classes))

In [10]:
print(x_train.shape, y_train.shape, y_train[0].shape, y_train.dtype)

(8,) (8, 1) (1,) int64


In [4]:


# y_train = np.random.randint(0, 2, (len(train), 44))
# y_valid = np.random.randint(0, 2, (len(valid), 44))
# y_test = np.random.randint(0, 2, (len(test), 44))
# y_test_labels = test['labels'].values

a = AugmentedImageSequenceFromNpy(x=x_train[:test_data_len], y=y_train[:test_data_len],
                                  batch_size=len(x_train[:test_data_len]),
                                  augmentations=AUGMENTATIONS_TRAIN)

for x, y in a:
    print(x.shape, y.shape)
    break

a.on_epoch_end()

if os.environ.get("SHOULD_TRAIN", "True") == "True":
#     histories = train(model, x_train=x_train,
#                       y_train=y_train,
#                       x_valid=x_valid,
#                       y_valid=y_valid,
#                       n_epochs=n_epochs,
#                       n_classes=n_classes,
#                       batch_size=batch_size,
#                       log_dir=log_dir,
#                       weight_path=weight_path)
    
    print(f'Split train: {len(x_train)}')
    print(f'Split valid: {len(x_valid)}')

    histories = []
    learn_rates = [0.001, 0.0001, 0.00001]
    metrics = [Accuracy(), Precision(), Recall(), F1Score(num_classes=n_classes, average='micro'),
               FBetaScore(num_classes=n_classes, beta=2.0, average='micro')]
    loss = 'binary_crossentropy'
    metric_to_monitor = 'val_loss'

    for learn_rate_num, learn_rate in enumerate(learn_rates):
        print(f'Training model on fold with learn_rate {learn_rate}')
        optimizer = Adam(lr=learn_rate)
        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

        verbosity = 0
        callbacks = [
            EarlyStopping(monitor=metric_to_monitor, patience=2, verbose=verbosity),
            ReduceLROnPlateau(monitor=metric_to_monitor, factor=0.5, patience=2, min_lr=0.000001),
            TensorBoard(log_dir, histogram_freq=1),
            ModelCheckpoint(weight_path, monitor=metric_to_monitor, save_weights_only=False, save_best_only=True,
                            verbose=verbosity)
        ]

        # Generators
        train_generator = AugmentedImageSequenceFromNpy(x=x_train, y=y_train, batch_size=batch_size,
                                                        augmentations=AUGMENTATIONS_TRAIN)

        valid_generator = AugmentedImageSequenceFromNpy(x=x_valid, y=y_valid, batch_size=batch_size,
                                                        augmentations=AUGMENTATIONS_TEST)

        history = model.fit_generator(generator=train_generator,
                                      epochs=n_epochs,
                                      steps_per_epoch=len(train_generator),
                                      callbacks=callbacks,
                                      validation_data=valid_generator, validation_steps=len(valid_generator),
                                      shuffle=True, verbose=1)
        histories.append(history)

    # Attempt to avoid memory leaks
    del train_generator
    del valid_generator
    gc.collect()
    
    

# if os.environ.get("SHOULD_PREDICT", "True") == "True":
#     pred_test_labels = predict(model=model, weight_dir=weight_dir, x=x_test, batch_size=batch_size, n_classes=n_classes)
#     clf_report = classification_report(y_test_labels, pred_test_labels, target_names=classes)
#     print(clf_report)

(8, 120, 120, 3) (8, 1)
Split train: 8
Split valid: 3
Training model on fold with learn_rate 0.001
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1 steps, validate for 1 steps
Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 26.4005 - accuracy: 0.1250 - precision: 0.0000e+00 - recall: 0.0000e+00 - f1_score: 0.5000 - fbeta_score: 0.3846 - val_loss: 396.2237 - val_accuracy: 0.6667 - val_precision: 0.6667 - val_recall: 1.0000 - val_f1_score: 0.8000 - val_fbeta_score: 0.9091
Epoch 2/100
1/1 [==============================] - 1s 797ms/step - loss: 2768.0317 - accuracy: 0.3750 - precision: 0.3750 - recall: 1.0000 - f1_score: 0.5455 - fbeta_score: 0.7500 - val_loss: 44.9131 - val_accuracy: 0.6667 - val_precision: 0.6667 - val_recall: 1.0000 - val_f1_score: 0.8000 - val_fbeta_score: 0.9091
Epoch 3/100
1/1 [==============================] - 1s 501ms/step - loss: 361.3700 - accuracy: 0.3

Epoch 9/100
1/1 [==============================] - 1s 664ms/step - loss: 14.5081 - accuracy: 0.3750 - precision: 0.3750 - recall: 1.0000 - f1_score: 0.5455 - fbeta_score: 0.7500 - val_loss: 3.0188 - val_accuracy: 0.6667 - val_precision: 0.6667 - val_recall: 1.0000 - val_f1_score: 0.8000 - val_fbeta_score: 0.9091
Epoch 10/100
1/1 [==============================] - 1s 773ms/step - loss: 12.7662 - accuracy: 0.3750 - precision: 0.5000 - recall: 1.0000 - f1_score: 0.6000 - fbeta_score: 0.7895 - val_loss: 2.7299 - val_accuracy: 0.6667 - val_precision: 0.6667 - val_recall: 1.0000 - val_f1_score: 0.8000 - val_fbeta_score: 0.9091
Epoch 11/100
1/1 [==============================] - 1s 665ms/step - loss: 9.5056 - accuracy: 0.3750 - precision: 0.3750 - recall: 1.0000 - f1_score: 0.5455 - fbeta_score: 0.7500 - val_loss: 2.4387 - val_accuracy: 0.6667 - val_precision: 0.6667 - val_recall: 1.0000 - val_f1_score: 0.8000 - val_fbeta_score: 0.9091
Epoch 12/100


KeyboardInterrupt: 